# P5 - Vehicle Detection - Model Training

# File sections:
* import section
* general utilities
* load the training data and data exporation
* HOG parameters selection by visualization
* Spacial parameters selection by visualization
* Color Hist parmeters selection

## imports for the whole project

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import pickle
import os
import matplotlib.image as mpimg
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from random import randint
import time
from sklearn.svm import LinearSVC
from IPython import display
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline

### imports for the whole project - end

## General Utility Functions - not directed related to the project

In [2]:
# Read Image

def readImages(dir, pattern = '*[.png][.jpg][.jepg]', cFormat='RGB'):
    # read all images with the given patterns (extensions) in the sub-directories
    # Example of a pattern: pattern = '*[.png][.jpg]'

    """
    Returns an image list with the image contained on the directory `dir` matching the `pattern`.
    pattern
    """
    images = []
    for imageFileName in glob.iglob(dir + '**/' + pattern, recursive=True):
        images.append(mpimg.imread(imageFileName))
    return images

### General Utility Functions - end

## Parameters definition - classes

# Value object to hold all feature extraction parameters.

In [3]:
class HogParameters():
    def __init__(self):
        # HOG parameters
        self.orient = 9
        self.pix_per_cell = 8
        self.cell_per_block = 2
        self.vis= False

class FeaturesParameters():
    def __init__(self):
        # consider different channels from differnet colors
        self.combined_channels = [['YCrCb',[0,1,2]]] #,['HLS',[0]]] # [['RGB',[0]], ['YCrCb',[0,2]],['HSV',[0]],['LUV',[0,1]]] # 6 channels! instead of 3'RGB' #'YCrCb' [['YCrCb',[0,1,2]]] #
        self.color_space = 'YCrCb' #[('RGB',0),('YCrCb',1),('HSV',0),('LUV',1),('LUV',1)] #'RGB' #'YCrCb'
        # Bin spatial parameters
        self.spatial_size = (32, 32)
        # Histogram parameters
        self.hist_bins = 32
        # HOG parameters
        self.hog_params=HogParameters()
        
class DetectionParameters():
    def __init__(self):
        self.ystart = 400
        self.ystop = 656
        self.scale = 1.5

Movie_History_length = 8


### Parameters definition - end

## Parameters extraction functions

# 3. Pipeline

## Features extraction and Model fitting: 

### Extract Features: Individual features

In [4]:
# Define a function to return HOG features and visualization
def convert_color(img, conv='RGB2YCrCb'):
    if conv == 'RGB2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if conv == 'BGR2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    if conv == 'RGB2LUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)

def color_convert (image, color_space='RGB'):
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else:
            print("Error undefined color sapace: ", color_space)
    else: 
        feature_image = np.copy(image)      
    return feature_image

def color_convert_combined (image, combined_channels=[['RGB',[0,1,2]]]):
    number_of_channels=0
    for color_sys in range (len(combined_channels)):
        feature_image = color_convert(image, combined_channels[color_sys][0])
        for channel in range (len(combined_channels[color_sys][1])):
            if number_of_channels==0:
                all_ch_img = feature_image[:,:,combined_channels[color_sys][1][channel]]
                all_ch_img = np.expand_dims(all_ch_img, axis=2)
            else:
                chanel = combined_channels[color_sys][1][channel]
                chanel_image = feature_image[:,:,combined_channels[color_sys][1][channel]]
                all_ch_img = np.insert(all_ch_img,0,chanel_image, axis=2)
            number_of_channels+=1
    return number_of_channels, all_ch_img

def get_hog_channel_features(feature_image, p, vis, feature_vec):
    return (hog(feature_image, orientations=p.orient, pixels_per_cell=(p.pix_per_cell,p.pix_per_cell),
                cells_per_block=(p.cell_per_block, p.cell_per_block), 
                visualise=vis, transform_sqrt=True, feature_vector=feature_vec))
# KZOH: transform_sqrt=True??

def get_hog_features(feature_image, p):
    hog_features=[]
    for channel in range(feature_image.shape[2]):
        hog_features.append(get_hog_channel_features(feature_image[:,:,channel], 
                            p, vis=False, feature_vec=True))
    hog_features = np.ravel(hog_features)   
    return (hog_features)

# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    colors = cv2.resize(img[:,:,img.shape[2]-1], size).ravel()
    for channel in range(img.shape[2]-1):
        nextColor = cv2.resize(img[:,:,img.shape[2]-channel-2], size).ravel()
        colors = np.hstack((colors, nextColor))
    return colors

# Define a function to compute color histogram features 
# NEED TO CHANGE bins_range if reading .png files with mpimg!

def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    hists = np.histogram(img[:,:,img.shape[2]-1], bins=nbins)[0]
    for channel in range(img.shape[2]-1):
            nextHist = np.histogram(img[:,:,img.shape[2]-channel-2], bins=nbins)[0] # , range=bins_range
            hists = np.concatenate(([hists, nextHist]))
    return hists

### Extract Features: Main function

In [5]:
# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()

# Parameters: color_space: 
# num of orientation bins (how many orientation. Normally 9)
# grid of the cells: how many cells in the image 
#cell size: pixcels / cel
# adding overlaps between cells
# block normalizarion
def extract_features(image, params):

    feature_image = color_convert(image, params.color_space)
    image_features = []

    # Apply bin_spatial() to get spatial color features
    spatial_features = bin_spatial(feature_image, size=params.spatial_size)

    # Apply color_hist()
    hist_features = color_hist(feature_image, nbins=params.hist_bins)

    # Apply hog() to get histogram orientation gradient features
    hog_features = get_hog_features(feature_image, params.hog_params)
    tmp_features = np.concatenate((spatial_features, hist_features, hog_features))
    
    return tmp_features

def extract_features_images(images, params):
    # Create a list to append feature vectors to
    features = []
    for img in images:
        imgFeatures = extract_features(img, params)
        features.append(imgFeatures)
    return features

### Fit Model

In [6]:
def fitModelSVC( positive_features, negative_features, params):
    
    # Create an array stack of feature vectors
    X = np.vstack((positive_features, negative_features)).astype(np.float64)

    # Define the labels vector
    y = np.hstack((np.ones(len(positive_features)), np.zeros(len(negative_features))))

    # Split up data into randomized training and test sets
    rand_state = np.random.randint(0, 100)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=rand_state)

    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X_train)
    # Apply the scaler to X
    X_train = X_scaler.transform(X_train)
    X_test = X_scaler.transform(X_test)

    # Use a linear SVC 
    svc = LinearSVC()
    # Check the training time for the SVC
    t=time.time()
    svc.fit(X_train, y_train)
    t2 = time.time()
    fittingTime = round(t2 - t, 2)
    
    accuracy = round(svc.score(X_test, y_test),4)
    
    print('Feature vector length:', len(X_train[0]))
    print('Fitting time:', fittingTime)
    print('Accuracy: ', accuracy)
    return (svc, X_scaler)

In [7]:
vehicles = readImages('./training_images/vehicles/vehicles/', '*.png')
non_vehicles = readImages('./training_images/non-vehicles/non-vehicles/', '*.png')
print ('Number of Vehicle images = ' + str(len(vehicles)))
print ('Number of non-Vehicle images = ' + str(len(non_vehicles)))

Number of Vehicle images = 8791
Number of non-Vehicle images = 8968


In [8]:
print (vehicles[0].shape)

(64, 64, 3)


In [9]:
params = FeaturesParameters()
vehicles_features = extract_features_images(vehicles, params)
non_vehicles_features = extract_features_images(non_vehicles, params)

svc, X_scaler = fitModelSVC(vehicles_features, non_vehicles_features, params)
pickle.dump( { 'svc': svc, 'X_scaler': X_scaler }, open('./model.p', 'wb'))

/home/carkyo1080-1/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


Feature vector length: 8460
Fitting time: 4.0
Accuracy:  0.9893


### Features extraction and Model fitting: end

## Test model on a random image (car and non_car)

In [36]:
vehicle_num = randint(0, len(vehicles)-1)
non_vehicle_num = randint(0, len(non_vehicles)-1)

test_vehicle = vehicles[vehicle_num]
test_non_vehicle = non_vehicles[non_vehicle_num]

In [37]:
dist_pickle = pickle.load( open( "./model.p", "rb" ) )
svc = dist_pickle["svc"]; 
X_scaler = dist_pickle["X_scaler"]

In [38]:
params = FeaturesParameters()
test_vehicle_features = extract_features(test_vehicle, params)
test_non_vehicle_features = extract_features(test_non_vehicle, params)

vehicle_features = X_scaler.transform(test_vehicle_features.reshape(1, -1))
non_vehicle_features = X_scaler.transform(test_non_vehicle_features.reshape(1, -1))

In [39]:
if (svc.predict(vehicle_features)) == 1:
    print ('Vehicle --> Prediction: Vehilce')
else:
    print('Vehicle --> Prediction: Non-Vehilce')

if (svc.predict(non_vehicle_features)) == 1:
    print('Non-Vehilce --> Prediction: Vehilce')
else:
    print('Non-Vehilce --> Prediction: Non-Vehilce')

Vehicle --> Prediction: Vehilce
Non-Vehilce --> Prediction: Non-Vehilce
